<a href="https://colab.research.google.com/github/ganeshsprofessional/sdc-GenAI/blob/main/kmeans_bookrecomender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
!pip install gradio scikit-learn pandas numpy joblib

import numpy as np
import pandas as pd
import joblib
import gradio as gr
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

# =======================
# 1️⃣ Load Data & Model
# =======================

# Re-load Data
df = pd.read_csv("shopping_data.csv")  # Make sure the correct file is used

# Load models safely
try:
    kmeans = joblib.load("kmeans_model.pkl")
    scaler = joblib.load("scaler.pkl")
except Exception as e:
    print("Error loading models:", str(e))

# Available genres
genres = ['Fiction', 'Mystery', 'Sci-Fi', 'Fantasy', 'Non-Fiction', 'Romance']

# =======================
# 2️⃣ Fix Gradio Function
# =======================
def recommend_books(rating, pages, year, genre):
    try:
        # Convert genre to numerical encoding
        genre_map = {g: i for i, g in enumerate(genres)}
        genre_encoded = genre_map.get(genre, 0)

        # Convert input to numpy array
        input_features = np.array([[rating, pages, year, genre_encoded]], dtype=np.float64)

        # Scale the features
        input_scaled = scaler.transform(input_features)

        # Predict cluster
        cluster = kmeans.predict(input_scaled)[0]

        # Find books in the same cluster
        recommended_books = df[df['cluster'] == cluster].sample(5)  # Pick 5 random books

        # Format the output
        recommendations = "\n".join(
            [f"📚 Book {row['book_id']} - {genres[int(row['genre_encoded'])]} ({int(row['num_pages'])} pages, {int(row['publication_year'])})"
             for _, row in recommended_books.iterrows()]
        )

        return recommendations

    except Exception as e:
        return f"⚠️ Error: {str(e)}"

# =======================
# 3️⃣ Launch Gradio UI
# =======================

interface = gr.Interface(
    fn=recommend_books,
    inputs=[
        gr.Slider(2.0, 5.0, step=0.1, label="Preferred Rating"),
        gr.Slider(100, 1000, step=50, label="Preferred Number of Pages"),
        gr.Slider(1950, 2023, step=1, label="Preferred Publication Year"),
        gr.Radio(genres, label="Preferred Genre")
    ],
    outputs="text",
    title="📖 AI Book Recommender",
    description="Enter your preferences to find books that match your interests!"
)

# Run the Gradio app
interface.launch()


FileNotFoundError: [Errno 2] No such file or directory: 'shopping_data.csv'